In [ ]:
# import general packages
import os
import copy
import time
import ntpath
import pickle
import warnings
import numpy as np
import pandas as pd

# import tkinter packages
import tkinter
from tkinter import filedialog
from tkinter import messagebox
from tkinter.ttk import Progressbar
from tkinter.ttk import Button

# import music analysis packages
import librosa
import pyAudioAnalysis as pyaudio
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures

In [ ]:
# toggle warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('default')

In [ ]:
# select the directory of the mp3 files using tkinter
tkobj = tkinter.Tk()
tkobj.withdraw()
tkobj.directory = filedialog.askdirectory(message = "Select the folder containing the MP3 files.")
path = tkobj.directory

In [ ]:
# start timer as soon as a directory is chosen
start = time.time()

In [ ]:
# code for progress button
# progress = Progressbar(tkobj, length = 100, mode = 'determinate')
# progress.pack()
# Button(tkobj, text = 'Generate Playlists').pack()
# tkobj.mainloop()
# progress['value'] = 20
# tkobj.update()

In [ ]:
# extract raw features
m3u_paths = {}
librosa_features = {}
pyaudio_features = {}
feat_names = None

# pyAudioAnalysis feature extraction variables
mid_term_window = 1
mid_term_step = 1
short_term_window = 0.05
short_term_step = 0.05

for root, dirs, files in os.walk(path, topdown = False):
    # extract pyAudioAnalysis features from the root folder
    pyaudio_feat, song_files, feat_names = MidTermFeatures.directory_feature_extraction(root, 
                                                                                        mid_term_window, 
                                                                                        mid_term_step, 
                                                                                        short_term_window, 
                                                                                        short_term_step,
                                                                                        False)
    # convert pyAudioAnalysis features into dictionary format
    counter = 0
    for s in song_files:
        s_dict_name = ntpath.basename(s)
        if pyaudio_feat.ndim == 1:
            pyaudio_features[s_dict_name] = pyaudio_feat
        else:
            pyaudio_features[s_dict_name] = pyaudio_feat[counter]
        counter += 1
    
    # extract pyAudioAnalysis features from subfolders
    for folder in dirs:
        folder_path = os.path.join(root, folder)
        pyaudio_feat, song_files, feat_names = MidTermFeatures.directory_feature_extraction(folder_path, 
                                                                                            mid_term_window, 
                                                                                            mid_term_step, 
                                                                                            short_term_window, 
                                                                                            short_term_step,
                                                                                            False)
        # convert pyAudioAnalysis features into dictionary format
        counter = 0
        for s in song_files:
            s_dict_name = ntpath.basename(s)
            if pyaudio_feat.ndim == 1:
                pyaudio_features[s_dict_name] = pyaudio_feat
            else:
                pyaudio_features[s_dict_name] = pyaudio_feat[counter]
            counter += 1
    
    
    # extract librosa features
    for song in files:
        song_path = os.path.join(root, song)
        if song_path.endswith(".mp3"):
            # get the tempo of the song
            waveform, samp_rate = librosa.load(song_path)
            tempo, beat_frames = librosa.beat.beat_track(waveform, samp_rate)

            # get the chroma number of the song
            beat_times = librosa.frames_to_time(beat_frames, samp_rate)
            y_harmonic, y_percussive = librosa.effects.hpss(waveform)
            chromagram = librosa.feature.chroma_cqt(y_harmonic, samp_rate)
            beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate = np.median)
            
            # make beat chroma into a DataFrame and calculate the diff for a single float
            chroma_df = pd.DataFrame(beat_chroma)
            diff_values = chroma_df.diff()
            diff_mean = diff_values.mean(axis = 0, skipna = True)
            chroma_num = sum(diff_mean) / len(diff_mean)
            
            # save librosa features and the path to each song
            librosa_features[song] = [tempo, chroma_num]
            m3u_paths[song] = song_path

In [ ]:
# show an error message if the feature extraction failed
if len(pyaudio_features) != len(librosa_features):
    messagebox.showinfo("Error", "Import of MP3 files failed.")

In [ ]:
# only keep desired features from pyAudioAnalysis for stacked classifier
new_pyaudio_features = {}
selected_feats = ['zcr_mean', 'zcr_std', 'energy_mean', 'energy_entropy_mean', 'spectral_centroid_mean', 
                  'spectral_spread_mean', 'spectral_entropy_mean', 'mfcc_2_mean', 'mfcc_5_mean', 'mfcc_6_mean',
                  'spectral_centroid_std', 'spectral_entropy_std', 'spectral_spread_std', 'chroma_7_std',
                  'delta chroma_2_std', 'delta chroma_3_std', 'delta chroma_9_std', 'delta chroma_std_std',
                  'delta energy_std', 'delta mfcc_1_std', 'delta mfcc_3_std', 'delta mfcc_13_std',
                  'delta spectral_centroid_std', 'delta spectral_entropy_std', 'delta spectral_flux_std',
                  'delta spectral_spread_std']
for key, value in pyaudio_features.items():
    index_list = [feat_names.index(feat) for feat in selected_feats]
    extracted_feats = [value[i] for i in index_list]
    new_pyaudio_features[key] = extracted_feats

In [ ]:
# transpose and merge the engineered features into one feature DataFrame
librosa_df = pd.DataFrame(librosa_features)
librosa_df = librosa_df.transpose()
eng_pyaudio_df = pd.DataFrame(new_pyaudio_features)
eng_pyaudio_df = eng_pyaudio_df.transpose()
eng_features = librosa_df.merge(eng_pyaudio_df, right_index = True, left_index = True)
eng_features.sort_index(inplace = True)

In [ ]:
# generate neural network features
pyaudio_df = pd.DataFrame(pyaudio_features)
pyaudio_df = pyaudio_df.transpose()
nn_features = pyaudio_df
nn_features.sort_index(inplace = True)

In [ ]:
# load the stacking classifier model and predict using the engineered features
loaded_stacker = pickle.load(open('mood_stacking_model.sav', 'rb'))
stacker_predictions = loaded_stacker.predict(eng_features)

In [ ]:
# load the mlp model and predict using the neural network features
loaded_mlp = pickle.load(open('mood_mlp_model.sav', 'rb'))
mlp_predictions = loaded_mlp.predict(nn_features)

In [ ]:
# combine predictions from both models for a final predictions list
final_preds = []

for i in range(len(stacker_predictions)):
    stack_pred = stacker_predictions[i]
    mlp_pred = mlp_predictions[i]
    
    if stack_pred != mlp_pred:
        # prediction was different between stacking and mlp models
        if stack_pred == 1 or stack_pred == 3:
            # prefer stacking classifier for moods 1 and 3
            final_preds.append(stack_pred)
        else:
            final_preds.append(mlp_pred)
    else:
        # predictions are the same, just append mlp's prediction
        final_preds.append(mlp_pred)

In [ ]:
# generate song lists based on predicted moods
# 1 epic, 2 lighthearted, 3 energetic, 4 calm, 5 chill, 6 miscellaneous
epic = []
lighthearted = []
energetic = []
calm = []
chill = []
miscellaneous = []

# ordering of indices are the same as engineered features due to prior sorting
for i in range(len(final_preds)):
    song = eng_features.index[i]
    pred = final_preds[i]
    if pred == 1:
        epic.append(song)
    if pred == 2:
        lighthearted.append(song)
    if pred == 3:
        energetic.append(song)
    if pred == 4:
        calm.append(song)
    if pred == 5:
        chill.append(song)
    if pred == 6:
        miscellaneous.append(song)

In [ ]:
# extract the relative paths of each song for m3u file format
m3u_paths_relative = {}
for key, value in m3u_paths.items():
    root_length = len(path) + 1
    m3u_paths_relative[key] = value[root_length:]

In [ ]:
# function to generate m3u files based on song lists
def write_playlist(path, filename, mood_list):
    full_path = path + "/" + filename + ".m3u"
    file = open(full_path, "w")
    playlist = [m3u_paths_relative[song] + "\n" for song in mood_list]
    playlist.insert(0, "#EXTM3U\n")
    file.writelines(playlist)
    file.close()

In [ ]:
# create the playlist files
write_playlist(path, "epic_playlist", epic)
write_playlist(path, "lighthearted_playlist", lighthearted)
write_playlist(path, "energetic_playlist", energetic)
write_playlist(path, "calm_playlist", calm)
write_playlist(path, "chill_playlist", chill)
write_playlist(path, "miscellaneous_playlist", miscellaneous)

In [ ]:
# stop the timer when files are done writing
end = time.time()
time_elapsed = round((end - start) / 60, 2)

In [ ]:
# display the time elapsed in a message box
message = "Total Time Elapsed: " + str(time_elapsed) + " minutes"
messagebox.showinfo("Playlists have been successfully created.", message)